In [32]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
    
colnames=['StockID', 'DateTime', 'PreClosePx', 'openPrice','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','b1','b2','b3','b4','b5','b6','b7','b8','b9','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b20',
          'b21','b22','b23','b24','b25','b26','b27','b28','b29','b30','b31','b32','b33','b34','b35','b36','b37','b38','b39','b40','b41','b42','b43',
          'b44','b45','b46','b47','b48','b49','b50','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14',
         'a15','a16','a17','a18','a19','a20','a21','a22','a23','a24','a25','a26','a27','a28','a29','a30','a31','a32','a33','a34','a35','a36','a37','a38',
         'a39','a40','a41','a42','a43','a44','a45','a46','a47','a48','a49','a50','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
colnames1=['StockID', 'DateTime', 'PreClosePx', 'openPrice','HighPx','LowPx','close','cum_volume','cum_amount','InstrumentStatus',
         'bid1p','bid2p','bid3p','bid4p','bid5p','bid6p','bid7p','bid8p','bid9p','bid10p','bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q',
         'bid8q','bid9q','bid10q','b1n', 'b2n', 'b3n', 'b4n', 'b5n', 'b6n', 'b7n', 'b8n', 'b9n', 'b10n','b1','b2','b3','b4','b5','b6','b7','b8','b9','b10','b11','b12','b13','b14','b15','b16','b17','b18','b19','b20',
          'b21','b22','b23','b24','b25','b26','b27','b28','b29','b30','b31','b32','b33','b34','b35','b36','b37','b38','b39','b40','b41','b42','b43',
          'b44','b45','b46','b47','b48','b49','b50','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p','ask1q','ask2q',
          'ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q','ask10q','a1n', 'a2n', 'a3n', 'a4n', 'a5n', 'a6n', 'a7n', 'a8n', 'a9n', 'a10n','a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14',
         'a15','a16','a17','a18','a19','a20','a21','a22','a23','a24','a25','a26','a27','a28','a29','a30','a31','a32','a33','a34','a35','a36','a37','a38',
         'a39','a40','a41','a42','a43','a44','a45','a46','a47','a48','a49','a50','NumTrades','IOPV','TotalBidQty','TotalOfferQty','WeightedAvgBidPx',
         'WeightedAvgOfferPx','TotalBidNumber','TotalOfferNumber','BidTradeMaxDuration','OfferTradeMaxDuration','NumBidOrders','NumOfferOrders','WithdrawBuyNumber',
         'WithdrawBuyAmount','WithdrawBuyMoney','WithdrawSellNumber','WithdrawSellAmount','WithdrawSellMoney','ETFBuyNumber','ETFBuyAmount','ETFBuyMoney',
         'ETFSellNumber','ETFSellAmount','ETFSellMoney'] 
snapshot = pd.read_csv("G:\\2018\\20180927\\Snapshot.csv", header=None, names=colnames1).iloc[1:, :]
snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
snapshot['time'] = ((snapshot["DateTime"] - int(snapshot["DateTime"].iloc[0]//1000000 * 1000000))*1000).astype(int)
snapshot["StockID"] = snapshot["StockID"].astype(int)
snapshot = snapshot[(snapshot["StockID"] >= 600000) & (snapshot["StockID"] < 700000)].reset_index(drop=True)
snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)

snapshot = snapshot.sort_values(by=["StockID", "DateTime"])   
data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "DateTime", "time"]]
del snapshot
data2 = data2.sort_values(by=["StockID", "DateTime"])
data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
data2["time1"] = pd.to_datetime(data2["DateTime"].astype(str))
data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
data2 = data2.rename(columns={"75%": "75%_time"})
vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
vu = vu.rename(columns={"volume_update":"90%"})
data2 = pd.merge(data2, vu, on="StockID")
data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
data2["indicator"] = 0
data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
data2 = data2[data2["indicator"] != 1]
data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)]

,StockID,cum_volume,DateTime,time,volume_update,time1,time_interval,75%_time,90%,indicator
1746075,600733,10085700.0,20180927100040,100040000,1899300.0,2018-09-27 10:00:40,1800.0,3.0,1040038.2,0


In [33]:
data2[data2["StockID"] == 600733]

,StockID,cum_volume,DateTime,time,volume_update,time1,time_interval,75%_time,90%,indicator
1746059,600733,4159400.0,20180927092501,92501000,NaN,2018-09-27 09:25:01,NaN,3.0,1040038.2,0
1746061,600733,6484356.0,20180927093004,93004000,1784961.0,2018-09-27 09:30:04,3.0,3.0,1040038.2,0
1746062,600733,6554706.0,20180927093007,93007000,70350.0,2018-09-27 09:30:07,3.0,3.0,1040038.2,0
1746063,600733,6591631.0,20180927093010,93010000,36925.0,2018-09-27 09:30:10,3.0,3.0,1040038.2,0
1746064,600733,6657881.0,20180927093013,93013000,66250.0,2018-09-27 09:30:13,3.0,3.0,1040038.2,0
1746065,600733,6816806.0,20180927093016,93016000,158925.0,2018-09-27 09:30:16,3.0,3.0,1040038.2,0
1746066,600733,6953756.0,20180927093019,93019000,136950.0,2018-09-27 09:30:19,3.0,3.0,1040038.2,0
1746067,600733,7052656.0,20180927093022,93022000,98900.0,2018-09-27 09:30:22,3.0,3.0,1040038.2,0
1746068,600733,7138225.0,20180927093025,93025000,85569.0,2018-09-27 09:30:25,3.0,3.0,1040038.2,0
1746069,600733,7198525.0,20180927093028,93028000,60300.0,2018-09-27 09:30:28,3.0,3.0,1040038.2,0


In [11]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SH' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs[:1]:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]


In [12]:
db["date"] = db["date"].str[:4] + db["date"].str[5:7] + db["date"].str[8:]
db["date"] = db["date"].astype(int)
db

,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,...,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
0,SH603693,20180703,10.80,12.96,10.80,12.96,9.00,1.000000,0.440000,0.440000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
1,SH603693,20180704,14.26,14.26,14.26,14.26,12.96,1.000000,0.100309,0.584444,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
2,SH603693,20180705,15.69,15.69,15.69,15.69,14.26,1.000000,0.100281,0.743333,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
3,SH603693,20180706,17.26,17.26,17.26,17.26,15.69,1.000000,0.100064,0.917778,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
4,SH603693,20180709,18.99,18.99,18.99,18.99,17.26,1.000000,0.100232,1.110000,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,SH603693,20200410,8.80,8.85,8.51,8.58,8.80,0.969676,-0.025000,0.003509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193000000.0,618000000.0
432,SH603693,20200413,8.54,8.57,8.42,8.53,8.58,0.969676,-0.005828,0.011862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193000000.0,618000000.0
433,SH603693,20200414,8.55,8.65,8.51,8.64,8.53,0.969676,0.012896,-0.024831,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193000000.0,618000000.0
434,SH603693,20200415,8.60,8.64,8.51,8.52,8.64,0.969676,-0.013889,-0.022936,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193000000.0,618000000.0


In [14]:
import pickle
logSH = pd.read_csv("F:\\SH.csv")
m_in = logSH['Date'].min()
m_ax = logSH['Date'].max()
date1 = logSH["Date"].unique()
date2 = db[(db["date"] >= m_in) & (db["date"] <= m_ax)]["date"].unique()
display(set(date2) - set(date1))
display(set(date1) - set(date2))

{20190307,
 20190313,
 20190410,
 20190411,
 20190412,
 20190415,
 20190416,
 20190417,
 20190619,
 20190620,
 20190621,
 20190624,
 20190625,
 20190626,
 20190930}

set()

In [15]:
np.sort(logSH["Date"].unique())

array([20190201, 20190211, 20190212, 20190213, 20190214, 20190215,
       20190218, 20190219, 20190220, 20190221, 20190222, 20190225,
       20190226, 20190227, 20190228, 20190301, 20190304, 20190305,
       20190306, 20190308, 20190311, 20190312, 20190314, 20190315,
       20190318, 20190319, 20190320, 20190321, 20190322, 20190325,
       20190326, 20190327, 20190328, 20190329, 20190401, 20190402,
       20190403, 20190404, 20190408, 20190409, 20190418, 20190419,
       20190422, 20190423, 20190424, 20190425, 20190426, 20190429,
       20190430, 20190506, 20190507, 20190508, 20190509, 20190510,
       20190513, 20190514, 20190515, 20190516, 20190517, 20190520,
       20190521, 20190522, 20190523, 20190524, 20190527, 20190528,
       20190529, 20190530, 20190531, 20190603, 20190604, 20190605,
       20190606, 20190610, 20190611, 20190612, 20190613, 20190614,
       20190617, 20190618, 20190627, 20190628, 20190701, 20190702,
       20190703, 20190704, 20190705, 20190708, 20190709, 20190